In [ ]:
import altair as alt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
iris = pd.read_csv('iris.csv')
iris.info()
iris.head(5)
iris.describe()

# Design decision 1: 散点图可视化iris特征
choice 1: 直接的散点图 choice 2: 回归散点图

In [ ]:
###回归散点图###
# f, axes = plt.subplots(3, 2, figsize=(15, 15))
# # fig = plt.figure(figsize=(15, 15), constrained_layout=False)
# # spec = gridspec.GridSpec(ncols=3, nrows=2)

f_1 = sns.lmplot(data=iris, x='SepalWidth', y='SepalLength', palette='RdPu', hue='Species')
f_2 = sns.lmplot(data=iris, x='SepalWidth', y='PetalLength', palette='RdPu', hue='Species')
f_3 = sns.lmplot(data=iris, x='SepalWidth', y='PetalWidth', palette='RdPu', hue='Species')
f_4 = sns.lmplot(data=iris, x='SepalLength', y='PetalLength', palette='RdPu', hue='Species')
f_5 = sns.lmplot(data=iris, x='SepalLength', y='PetalWidth', palette='RdPu', hue='Species')
f_6 = sns.lmplot(data=iris, x='PetalLength', y='PetalWidth', palette='RdPu', hue='Species')


# Design Decision 2: 核密度估计图
choice 1: 单属性kde choice 2: 双属性kde

In [ ]:
f, axes = plt.subplots(3, 2, figsize=(10, 10))

swsl = sns.kdeplot(data=iris, x='SepalWidth',  y='SepalLength', palette='nipy_spectral_r', hue='Species', ax = axes[0,0])
swpl = sns.kdeplot(data=iris, x='SepalWidth',  y='PetalLength', palette='nipy_spectral_r', hue='Species', ax = axes[1,0])
swpw = sns.kdeplot(data=iris, x='SepalWidth',  y='PetalWidth',  palette='nipy_spectral_r', hue='Species', ax = axes[2,0])
slpl = sns.kdeplot(data=iris, x='SepalLength', y='PetalLength', palette='nipy_spectral_r', hue='Species', ax = axes[0,1])
slpw = sns.kdeplot(data=iris, x='SepalLength', y='PetalWidth',  palette='nipy_spectral_r', hue='Species', ax = axes[1,1])
plpw = sns.kdeplot(data=iris, x='PetalLength', y='PetalWidth',  palette='nipy_spectral_r', hue='Species', ax = axes[2,1])

# Design Decision 3: 是否合并箱线图和琴图

In [ ]:
###箱线图###
f, axes = plt.subplots(2, 2, figsize=(8, 8), sharex=True)
sns.despine(left=True)

sns.boxplot(x='Species', y='SepalLength', data=iris, palette='cool', ax=axes[0, 0])
sns.boxplot(x='Species', y='SepalWidth', data=iris, palette='cool', ax=axes[0, 1])
sns.boxplot(x='Species', y='PetalLength', data=iris, palette='cool', ax=axes[1, 0])
sns.boxplot(x='Species', y='PetalWidth', data=iris, palette='cool', ax=axes[1, 1])

plt.show()

# Design Decision 4: Andrew曲线 或者 Parallel_coordinates

In [ ]:
plt.subplots(figsize = (10,8))
pd.plotting.parallel_coordinates(iris, 'Species', colormap='cividis')
plt.show()

In [ ]:
####################################Not In Decisions#########################################

# 密度预估曲线图

In [ ]:
alt.Chart(iris).transform_fold(
    ['PetalWidth',
     'PetalLength',
     'SepalWidth',
     'SepalLength'],
    as_ = ['Measurement_type', 'value']
).transform_density(
    density='value',
    bandwidth=0.3,
    groupby=['Measurement_type'],
    extent= [0, 8],
    counts = True,
    steps=200
).mark_area().encode(
    alt.X('value:Q'),
    alt.Y('density:Q', stack='zero'),
    alt.Color('Measurement_type:N')
).properties(width=400, height=100)

In [ ]:
alt.Chart(iris).mark_area(interpolate='monotone').encode(
    x='SepalLength:Q',
    y='count(SepalLength):Q',
    color='Species:N',
    facet=alt.Facet('Species:N', title=None)
).properties(width=200, height=150)

In [ ]:
alt.Chart(iris).mark_area(interpolate='monotone').encode(
    x='SepalLength:Q',
    y='count(SepalLength):Q',
    color='Species:N',
    facet=alt.Facet('Species:N', title=None)
).transform_lookup(
    lookup='PetalLength',
    from_=alt.LookupData(data=iris, key='PetalLength',
                         fields=['PetalLength', 'SepalLength'])
).transform_filter(
    alt.FieldRangePredicate(field='SepalLength', range=[0, 100])
).properties(width=200, height=150)